In [1]:
import os,sys, copy, pickle

In [2]:
import pandas as pd

In [3]:
from streamm import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
res_local = resource.Resource('local')
res_local.load_json()

Reading in resource json file res_local.json 


In [6]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Find sub list of particle keys to be included in et calculation

In [7]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples'

In [8]:
p3ht_n10_x20_nvt = buildingblock.Container('p3ht_n10_x20_nvt')
p3ht_n10_x20_nvt.read_cply()

In [ ]:
p3ht_n10_x20_nvt.group_prop('residue','residue')

In [ ]:
p3ht_n10_x20_nvt.propcompile_particles()

In [ ]:
part_df =  pd.DataFrame(p3ht_n10_x20_nvt.prop_particles)

In [ ]:
syC = part_df['resname'] == 'SCP2'
syS = part_df['resname'] == 'ThS'

In [ ]:
sub_th = part_df[ syC | syS ]

In [ ]:
struc_th = p3ht_n10_x20_nvt.getSubStructure(sub_th.index,'sub_th') 

In [ ]:
struc_th.write_xyz()

Make sure the selection is correct by viewing the structure 

In [ ]:
xO = 20

In [ ]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [ ]:
lmp_bulk.load_json()

In [ ]:
os.getcwd()

In [ ]:
equ_et = calculation.CalculationRes('%s_et1'%(lmp_bulk.tag))

In [ ]:
equ_et.set_resource(peregrine)

In [ ]:
equ_et.properties['scratch'] = equ_et.dir['scratch']

In [ ]:
equ_et.add_refcalc(lmp_bulk)

In [ ]:
equ_et.make_dir()

In [ ]:

os.chdir(equ_et.dir['launch'])

In [ ]:
file_type = 'input'
file_key = 'cply'
file_name = "p3ht_n10_x20_nvt.cply"
from_dirkey = 'home'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "streamm_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 't_nwproj'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 't_nw'
file_name = "nwchem_et.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 't_run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
equ_et.dir['scripts'] = '/Users/tkemper/Development/streamm_ext/streamm-tools/scripts'
print equ_et.dir['scripts']

In [ ]:
file_type = 'scripts'
file_key = 'run_proj'
file_name = "run_proj.py"
from_dirkey = 'scripts'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'scripts'
file_key = 'py_script'
file_name = "et1.py"
from_dirkey = 'scripts'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
equ_et.properties['finish_str'] ='Finished'

In [ ]:
list_th = [str(pkey) for pkey in sub_th.index]

In [ ]:
outfile = open('list_th','wb')
outfile.write("\n".join(list_th))
outfile.close()

In [ ]:
equ_et.add_file('input','list_i','list_th')

In [ ]:
equ_et.properties['streamm_command'] ='python %s  %s --cply %s --list_i %s --t_nw %s --t_run %s '%(equ_et.files['scripts']['py_script'],equ_et.tag,equ_et.files['input']['cply'],equ_et.files['input']['list_i'],equ_et.files['templates']['t_nw'],equ_et.files['templates']['t_run'])
print equ_et.properties['streamm_command']

In [ ]:
equ_et.load_str('templates','run')

In [ ]:
equ_et.replacewrite_prop('run','scripts','run','%s.pbs'%(equ_et.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(equ_et.tag)
equ_et.add_file(file_type,file_key,file_name)

In [ ]:
equ_et.push()

In [ ]:
equ_et.run()